In [1]:
#import functions as func
import netCDF4
import numpy as np
import matplotlib.pyplot as plt
import os
import matplotlib as mpl
import matplotlib.colors as colors
import time
import math
from matplotlib.widgets import Slider, Button, RadioButtons
#from SlideView import SlideView
import seaborn as sns
import time
import locale
import analysis
#%matplotlib

Using matplotlib backend: Qt5Agg


In [3]:
# Loading weather state data (histograms)
pctaulist = []
tmphist = []
count = 0
with open('../code/pc_tau_oldData.txt') as pctauf:
    for line in pctauf:
        for value in line.split():
            tmphist.append(float(value))
            count += 1
        if count == 42:
            count = 0 
            pctaulist.append(np.asarray(tmphist).reshape(7, 6))
            tmphist = []

pctau_histograms = np.asarray(pctaulist)

# Transform pctau_histograms to tau_pc to match pctaudist (matrix transpose)
taupc_histograms = np.copy(pctau_histograms).reshape((12, 6, 7))
for i in range(pctau_histograms.shape[0]):
    taupc_histograms[i,:,:] = pctau_histograms[i,:,:].T
    
taupc_histograms = taupc_histograms.astype(np.float32)

In [9]:
# Loading file from harddrive
f = netCDF4.Dataset('/home/derektropf/Documents/NASA/code/ISCCP_HSeries_HGG_v01r00_198307_c20170705/ISCCP.HGG.v01r00.GLOBAL.1983.07.01.0000.GPC.10KM.CS00.EQ1.00.nc')
pctaudist = f['n_pctaudist']
sqlon_beg = f['sqlon_beg']
sqlon_end = f['sqlon_end']
eqlat = f['eqlat_index']
ntotal = f['n_total']

In [4]:
wsnums = np.zeros((360, 180, 12))
wscount = np.zeros((360, 180))
masks = np.zeros((6, 7, 41252), dtype=bool)
#basepath = '/home/derektropf/Documents/NASA/code/ISCCP_HSeries_HGG_v01r00_198307_c20170705'
#fullpath = '/home/derektropf/Documents/NASA/code/ISCCP_HSeries_HGG_v01r00_198307_c20170705/ISCCP.HGG.v01r00.GLOBAL.1983.07.01.0000.GPC.10KM.CS00.EQ1.00.nc'
basepath = '/media/derektropf/My Passport/hggdata/'
for f1 in sorted(os.listdir(basepath)):
    t1 = time.time()
    print('this is f1', f1)
    for f2 in sorted(os.listdir(basepath+f1+'/')):
        #print('this is f2', f2)
        #fullpath = basepath+'/'+f
        fullpath = basepath+f1+'/'+f2
        data = netCDF4.Dataset(fullpath)
        pctaudist = data['n_pctaudist']
        sqlonbeg = data['sqlon_beg'][:]
        sqlonend = data['sqlon_end'][:]
        eqlat = data['eqlat_index'][:]
        ntotal = data['n_total'][:]
        masks[:,:,:] = np.ma.getmaskarray(pctaudist[:,:,:])
        pctaudistnorm = analysis.fnorm(pctaudist[:,:,:], ntotal[:], masks[:,:,:])
        histograms = taupc_histograms
        wsnums, wscount = analysis.findws(pctaudistnorm[:,:,:], histograms[:,:,:]/100.0, masks[:,:,:], 
                                            wsnums[:,:], wscount[:], sqlonbeg[:], sqlonend[:], eqlat[:])
    t2 = time.time()
    print('run time for entire month ', t2-t1)

('this is f1', 'ISCCP_HSeries_HGG_v01r00_198501_c20170621')
('run time for entire month ', 80.34166502952576)
('this is f1', 'ISCCP_HSeries_HGG_v01r00_198502_c20170621')
('run time for entire month ', 77.31749105453491)
('this is f1', 'ISCCP_HSeries_HGG_v01r00_198503_c20170621')
('run time for entire month ', 84.92647194862366)
('this is f1', 'ISCCP_HSeries_HGG_v01r00_198504_c20170621')
('run time for entire month ', 82.06987690925598)
('this is f1', 'ISCCP_HSeries_HGG_v01r00_198505_c20170621')
('run time for entire month ', 87.26073098182678)
('this is f1', 'ISCCP_HSeries_HGG_v01r00_198506_c20170621')
('run time for entire month ', 83.2678451538086)
('this is f1', 'ISCCP_HSeries_HGG_v01r00_198507_c20170621')
('run time for entire month ', 81.46025085449219)
('this is f1', 'ISCCP_HSeries_HGG_v01r00_198508_c20170621')
('run time for entire month ', 81.62223100662231)
('this is f1', 'ISCCP_HSeries_HGG_v01r00_198509_c20170621')
('run time for entire month ', 82.56855416297913)
('this is f

In [37]:
w = np.where(wscount<=0)
mask = np.zeros((360, 180))
for point in zip(w[0], w[1]):
    mask[point[0], point[1]] = 1
arr = np.ma.array(wsnums[:,:,6], mask=mask, dtype=float)

In [38]:
fig, ax = plt.subplots()
sns.heatmap(np.flipud((arr/wscount).T))

In [23]:
sns.heatmap(np.flipud(wscount[:,:].T))